In [1]:
import pdb

In [1]:
import sympy
import copy
#import mpmath
#from mpmath import mp
import sympy.mpmath as mpmath
from sympy.mpmath.libmp.libhyper import NoConvergence
from sympy.mpmath import mp 
from sympy import poly
from sympy import symbols

print sympy.__version__
print mpmath.__version__

precision = 20
mp.dps = precision

print mp

0.7.6
0.18
Mpmath settings:
  mp.prec = 70                [default: 53]
  mp.dps = 20                 [default: 15]
  mp.trap_complex = False     [default: False]


In [2]:
x, z = symbols('x, z')

In [3]:
f_sym_eq = sympy.sympify("x**4 + a*z*x**2 + b*x + c*z**2")
parameters = {'a': -10, 'b': 4, 'c': 9}

In [4]:
f_sym_eq

a*x**2*z + b*x + c*z**2 + x**4

In [5]:
subs_dict = copy.deepcopy(parameters)

In [21]:
D_z_sym = sympy.poly(sympy.discriminant(f_sym_eq, x), z)
D_z_sym

Poly((16*a**4*c - 128*a**2*c**2 + 256*c**3)*z**6 + (-4*a**3*b**2 + 144*a*b**2*c)*z**3 - 27*b**4, z, domain='ZZ[a,b,c]')

In [7]:
cs = [c_sym.evalf(subs=subs_dict, n=precision).as_real_imag() for c_sym in D_z_sym.all_coeffs()]
for i, c_sym in enumerate(D_z_sym.all_coeffs()):
    print '{} = {}'.format(c_sym, cs[i])

16*a**4*c - 128*a**2*c**2 + 256*c**3 = (589824.00000000000000, 0)
0 = (0, 0)
0 = (0, 0)
-4*a**3*b**2 + 144*a*b**2*c = (-143360.00000000000000, 0)
0 = (0, 0)
0 = (0, 0)
-27*b**4 = (-6912.0000000000000000, 0)


In [8]:
D_z_coeffs = [mpmath.mpc(*c) for c in cs]

In [9]:
# Due to the bug of mpmath, error estimation does not work for version < 0.20-git
#D_z_roots, z_roots_error = mpmath.polyroots(D_z_coeffs, extraprec=precision, error=True)
#print 'estimated error of z roots: {}'.format(z_roots_error)

In [10]:
D_z_roots = mpmath.polyroots(D_z_coeffs, extraprec=precision)

In [20]:
print D_z_roots[0]

-0.34544520170447289579


In [12]:
for i, z_i in enumerate(D_z_roots):
    print 'D(z_{}) = {}'.format(i, abs(mpmath.polyval(D_z_coeffs, z_i)))

D(z_0) = 0.0
D(z_1) = 6.2450045135165055399e-17
D(z_2) = 1.4168545436079111663e-17
D(z_3) = 1.4168545436079111663e-17
D(z_4) = 9.6155545410899194482e-17
D(z_5) = 9.6155545410899194482e-17


In [17]:
abs(D_z_roots[0] - D_z_roots[1])

mpf('1.0029737271645977072129')

In [15]:
for i, z_i in enumerate(D_z_roots):
    print 'z_{} = {}\n'.format(i, z_i)
    subs_dict[z] = z_i
    
    f_x = sympy.poly(f_sym_eq.evalf(subs=subs_dict, n=precision), x)
    f_x_cs = [c.evalf(subs=subs_dict, n=precision) for c in sympy.poly(f_sym_eq, x).all_coeffs()]
    f_x_coeffs =  [mpmath.mpc(*c.as_real_imag()) for c in f_x_cs]
    #f_x_roots, x_roots_error = mpmath.polyroots(f_x_coeffs, maxsteps=50, extraprec=30, error=True)
    #print 'estimated error of x roots: {}\n'.format(x_roots_error)
    f_x_roots = None
    polyroots_maxsteps = 50
    polyroots_extra_precision = 30
    while f_x_roots is None:
        try:
            f_x_roots = mpmath.polyroots(f_x_coeffs, maxsteps=polyroots_maxsteps, extraprec=polyroots_extra_precision)
        except NoConvergence:
            print 'increase parameters...'
            polyroots_maxsteps += 10
            polyroots_extra_precision +=10
    for j, x_j in enumerate(f_x_roots):
        print '\tx_{} = {}'.format(j, x_j)
        print '\tf(z_{}, x_{}) = {}\n'.format(i, j, abs(mpmath.polyval(f_x_coeffs, x_j)))
    

z_0 = -0.34544520170447289579

	x_0 = -0.50458391000626530875
	f(z_0, x_0) = 0.0

	x_1 = -0.50458390999500690525
	f(z_0, x_1) = 0.0

	x_2 = (0.50458391000063610701 - 1.9908947389321689187j)
	f(z_0, x_2) = 1.0473773797765169395e-20

	x_3 = (0.50458391000063610701 + 1.9908947389321689187j)
	f(z_0, x_3) = 1.0473773797765169395e-20

z_1 = 0.65752852546012481143

	x_0 = -2.7416415395448845372
	f(z_1, x_0) = 2.3716922523120409494e-20

	x_1 = -0.5302892369343576246
	f(z_1, x_1) = 0.0

	x_2 = (1.6359653882396210809 - 1.3147268817715335806e-11j)
	f(z_1, x_2) = 3.3881317890172013563e-21

	x_3 = (1.6359653882396210809 + 1.3147268832853956146e-11j)
	f(z_1, x_3) = 3.3881317890172013563e-21

z_2 = (0.17272260085223644789 + 0.29916432029151299303j)

	x_0 = (0.25229195498544071555 - 0.43698248438724044883j)
	f(z_2, x_0) = 8.4703294725430033907e-22

	x_1 = (0.25229195501519539146 - 0.43698248441562301991j)
	f(z_2, x_1) = 0.0

	x_2 = (-1.9764573751763642059 - 0.55846488506465272499j)
	f(z_2, x_2) = 8.63